In [31]:
import sys, os
import rasterio, fiona

import pandas as pd
import geopandas as gpd

sys.path.insert(0, '../../src')
import GOSTrocks.rasterMisc as rMisc
from GOSTrocks.misc import tPrint

In [37]:
base_folder = r"C:\WBG\Work\Projects\COL_forests"
input_folder = os.path.join(base_folder, "INPUT")
forests_folder = os.path.join(input_folder, "Forest Data", "Mapbiomas")

output_folder = os.path.join(base_folder, "OUTPUT")
zonal_output_folder = os.path.join(output_folder, "Zonal Statistics")
if not os.path.exists(zonal_output_folder):
    os.makedirs(zonal_output_folder)

# Identify shape datasets
shape_files = {}
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".shp") or file.endswith(".kml") or file.endswith(".geojson"):
            shapefile = os.path.join(root, file)
            shape_group = os.path.basename(root)
            if shape_group not in shape_files:
                shape_files[shape_group] = []
            shape_files[shape_group].append(shapefile)

In [25]:
aoi_files = []

for key, items in shape_files.items():
    print(f"{key}:")
    out_file = os.path.join(output_folder, f"{key}.geojson")
    if not os.path.exists(out_file):        
        all_shapes = []
        for item in items:
            try:
                for layer in fiona.listlayers(item):
                    cur_shp = gpd.read_file(item, layer=layer)
                    cur_shp["source"] = key
                    cur_shp["source_file"] = os.path.basename(item)
                    cur_shp['geometry'] = cur_shp['geometry'].buffer(0)
                    all_shapes.append(cur_shp)
            except:
                print(f"Error reading {key}: {item}")
                continue
        all_shapes = pd.concat(all_shapes)
        all_shapes = all_shapes.reset_index(drop=True)        
        all_shapes.to_file(out_file, driver="GeoJSON", index=False)
    aoi_files.append(out_file)
        


BioCarbon:
Cercarbono:
Colcx:
Colombia Municipalities:
Verra:


In [30]:
# Define LC categories
lc_defs = {
    "All_Forest": {
        1: "All_forest",
        3: "Forest", 
        5: "Mangrove", 
        6: "Flooded Forest",
        49: "Wood sand vegetation",
    },
    "All_Natural non-forest": {
        10: "All_Natural non-forest",
        11: "Wetland",
        12: "Grassland",
        32: "Tidal flat",
        29: "Rocky outcrop",
        50: "Herbaceous sand vegetation",
        13: "Other non-forest",
    },
    "All_Agriculture": {
        14: "All_Agriculture",
        9: "Forest plantation",
        35: "Palm oil plantation",
        21: "Mosiac agriculture",
    },
    "All_non_vegetated": {  
        22: "All_non_vegetated", 
        23: "Beach",
        24: "Infrastructure",
        30: "Mining",
        68: "Other natural",
        25: "Other non_vegetated",
    },
    "All_Water": {
        26: "All_Water",
        33: "River, lake, ocean",
        31: "Aquaculture",
        34: "Glacier",
        27: "No Data"
    }
}

# Generate a list of unique land cover values
lc_unq = []
for lc_label, items in lc_defs.items():
    for key, value in items.items():
        lc_unq.append(key)

In [24]:
lc_tiffs = []
for root, dirs, files in os.walk(forests_folder):
    for file in files:
        if file.endswith(".tif") or file.endswith(".tiff"):
            lc_tiff = os.path.join(root, file)
            lc_tiffs.append(lc_tiff)
lc_tiffs

['C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2000.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2001.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2002.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2003.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2004.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2005.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2006.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2007.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\\Forest Data\\Mapbiomas\\Tif\\mapbiomas_colombia_2008.tif',
 'C:\\WBG\\Work\\Projects\\COL_forests\\INPUT\

In [40]:
for lc_tiff in lc_tiffs:
    cur_tiff = rasterio.open(lc_tiff)
    all_res = []
    for aoi_file in aoi_files:
        tPrint(f"Processing {os.path.basename(aoi_file)} with {os.path.basename(lc_tiff)}")
        aoi = gpd.read_file(aoi_file)
        if aoi.crs != cur_tiff.crs:
            aoi = aoi.to_crs(cur_tiff.crs)
        res = rMisc.zonalStats(aoi, cur_tiff, rastType='C', unqVals=lc_unq)
        res = pd.DataFrame(res, columns=[f'c{x}' for x in lc_unq])
        for col in res.columns:
            aoi[col] = res[col].values
        all_res.append(aoi)
    cur_year_res = pd.concat(all_res)
    cur_year_res.to_csv(os.path.join(zonal_output_folder, f"{os.path.basename(lc_tiff)}_zonal.csv"), index=False)
    
        

18:26:34	Processing BioCarbon.geojson with mapbiomas_colombia_2000.tif
18:26:42	Processing Cercarbono.geojson with mapbiomas_colombia_2000.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
  return ogr_read(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs

18:26:49	Processing Colcx.geojson with mapbiomas_colombia_2000.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

18:26:51	Processing Colombia Municipalities.geojson with mapbiomas_colombia_2000.tif
18:28:23	Processing Verra.geojson with mapbiomas_colombia_2000.tif
18:29:16	Processing BioCarbon.geojson with mapbiomas_colombia_2001.tif
18:29:24	Processing Cercarbono.geojson with mapbiomas_colombia_2001.tif


...

19:35:01	Processing Colombia Municipalities.geojson with mapbiomas_colombia_2022.tif
19:36:33	Processing Verra.geojson with mapbiomas_colombia_2022.tif
19:37:51	Processing BioCarbon.geojson with mapbiomas_colombia_2023.tif
19:37:58	Processing Cercarbono.geojson with mapbiomas_colombia_2023.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
  return ogr_read(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs

19:38:03	Processing Colcx.geojson with mapbiomas_colombia_2023.tif


c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\features.py:336: ShapeSkipWarning: Invalid or empty shape {'type': 'Polygon', 'coordinates': ()} at index 0 will not be rasterized.
  warnings.warn(
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")
c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\rasterio\fea

19:38:05	Processing Colombia Municipalities.geojson with mapbiomas_colombia_2023.tif
19:39:36	Processing Verra.geojson with mapbiomas_colombia_2023.tif


In [36]:
all_res[0]

,Name,Description,source,source_file,geometry,c1,c3,c5,c6,c49,...,c23,c24,c30,c68,c25,c26,c33,c31,c34,c27
0,Monterrey,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",BioCarbon,BCR-CO-173-14-001.kml,"MULTIPOLYGON Z (((-74.90459 9.67208 0, -74.905...",0,53922,0,380,0,...,0,16,37,0,36,0,782,0,0,0
1,Punto Nuevo,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",BioCarbon,BCR-CO-173-14-001.kml,"POLYGON Z ((-73.91527 9.84866 0, -73.91513 9.8...",0,601,0,216,0,...,0,0,0,0,0,0,0,0,0,0
2,,,BioCarbon,BCR-CO-259-14-002.kml,"POLYGON ((-71.69321 3.66253, -71.6928 3.6628, ...",0,9,0,19,0,...,0,0,0,0,0,0,0,0,0,0
3,,,BioCarbon,BCR-CO-259-14-002.kml,"MULTIPOLYGON (((-71.66731 3.65838, -71.66726 3...",0,3,0,38,0,...,0,0,0,0,0,0,0,0,0,0
4,,,BioCarbon,BCR-CO-259-14-002.kml,"POLYGON ((-71.71204 3.64704, -71.71237 3.64713...",0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,Lote 6,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",BioCarbon,PCR-CO-JCR-14-001.kml,"POLYGON Z ((-76.54741 8.68028 0, -76.54731 8.6...",0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006,Lote 6,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",BioCarbon,PCR-CO-JCR-14-001.kml,"POLYGON Z ((-76.54706 8.68031 0, -76.54731 8.6...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007,Lote 8a,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",BioCarbon,PCR-CO-JCR-14-001.kml,"POLYGON Z ((-76.54637 8.67625 0, -76.54668 8.6...",0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008,Lote 8a,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...",BioCarbon,PCR-CO-JCR-14-001.kml,"POLYGON Z ((-76.54771 8.67705 0, -76.54769 8.6...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
